In [5]:
import netgen.gui
from ngsolve import *
import netgen.geom2d as geom2dim

In [6]:
# CUSTOM geometry
"""geo = geom2dim.SplineGeometry()

vertices = [(0,0), (1,0), (1,1), (0,1)]
p1, p2, p3, p4 = [geo.AppendPoint(x,y) for x,y in vertices]
#lines = [['line', start, end, 0] for start, end in [(p1, p2), (p2, p3), (p3, p4), (p4, p5), (p5, p6), (p6,p1)]]
# Dirichlet bc
# Right
geo.Append(['line', p2, p3], bc=1)
# Left
geo.Append(['line', p4, p1], bc=1)

# Neumann bc
# Bot
geo.Append(['line', p1, p2], bc=2)
# Top
geo.Append(['line', p1, p2], bc=2)"""

"geo = geom2dim.SplineGeometry()\n\nvertices = [(0,0), (1,0), (1,1), (0,1)]\np1, p2, p3, p4 = [geo.AppendPoint(x,y) for x,y in vertices]\n#lines = [['line', start, end, 0] for start, end in [(p1, p2), (p2, p3), (p3, p4), (p4, p5), (p5, p6), (p6,p1)]]\n# Dirichlet bc\n# Right\ngeo.Append(['line', p2, p3], bc=1)\n# Left\ngeo.Append(['line', p4, p1], bc=1)\n\n# Neumann bc\n# Bot\ngeo.Append(['line', p1, p2], bc=2)\n# Top\ngeo.Append(['line', p1, p2], bc=2)"

In [40]:
#number_of_vertices = 4
mesh_size = 2e-1
mesh = Mesh(geom2dim.unit_square.GenerateMesh(maxh=mesh_size))
print("Number of vertices: ", mesh.nv)
print("Number of elements: ", mesh.ne)
Draw(mesh)

Number of vertices:  39
Number of elements:  56


In [50]:
fes = H1(mesh, order=3, dirichlet='right|left')

# Set trial and test function
u = fes.TrialFunction()
v = fes.TestFunction()

# Set right hand side
rhs = LinearForm(fes)
rhs += x*v*dx

# The bilinear form
A = BilinearForm(fes, symmetric=True)
A += grad(u)*grad(v)*dx

# Now assemble the system of equations
with TaskManager():
    rhs.Assemble()
    A.Assemble()

# Calculate the solution field (function)
gf = GridFunction(fes)
gf.vec.data = A.mat.Inverse(fes.FreeDofs(), inverse='sparsecholesky') * rhs.vec

# Calculate temperature flux through left and right boundaries
dudx = GridFunction(fes)
dudx.Set(grad(gf)[0])
dudy = GridFunction(fes)
dudy.Set(grad(gf)[1])
print("dudx: ", dudx)
#print(len(dudx.data))
print("dudy: ", dudy)

Draw(gf, mesh, 'grid_function')
Draw(grad(gf), mesh, 'gradient_gf')

dudx:  gridfunction 'gfu' on space 'H1HighOrderFESpace(h1ho)'
nested = 0

dudy:  gridfunction 'gfu' on space 'H1HighOrderFESpace(h1ho)'
nested = 0



In [59]:
# Analytical (exact) values
Wr_exact = -1/3
Wl_exact = 1/6
Wd_exact = Wr_exact + Wl_exact

#Wl = Integrate(dudx, mesh, BND, region_wise=True)
#print("Wl", Wl)
Wl = Integrate(dudx, mesh, definedon=mesh.Boundaries('left'))
print("Wl=", Wl)
Wr = Integrate(dudx, mesh, definedon=mesh.Boundaries('right'))
print("Wr=", Wr)
Wd = Wl + Wr

print('-'*30)
print("Total flux through dirichlet boundaries:\nWd= ", Wd)
print("Analytical (exact) value:")
print("Wl_exact=", Wl_exact)
print("Wr_exact=", Wr_exact)
print("Wd_exact=", Wd_exact)

Wl= 0.16666666666666496
Wr= -0.3333333333333297
------------------------------
Total flux through dirichlet boundaries:
Wd=  -0.16666666666666474
Analytical (exact) value:
Wl_exact= 0.16666666666666666
Wr_exact= -0.3333333333333333
Wd_exact= -0.16666666666666666


In [25]:
#help(fes)
help(Integrate)

Help on built-in function Integrate in module ngsolve.comp:

Integrate(...) method of builtins.PyCapsule instance
    Integrate(*args, **kwargs)
    Overloaded function.
    
    1. Integrate(cf: ngsolve.fem.CoefficientFunction, mesh: ngsolve.comp.Mesh, VOL_or_BND: ngsolve.comp.VorB = VorB.VOL, order: int = 5, definedon: ngsolve.comp.Region = None, region_wise: bool = False, element_wise: bool = False) -> object
    
    
    Parameters
    ----------
    
    cf: ngsolve.CoefficientFunction
      Function to be integrated. Can be vector valued, then the result is an array. If you want to integrate
      a lot of functions on the same domain, it will be faster to put them into a vector valued function,
      NGSolve will then be able to use parallelization and SIMD vectorization more efficiently.
    
    mesh: ngsolve.Mesh
      The mesh to be integrated on.
    
    VOL_or_BND: ngsolve.VorB = VOL
      Co-dimension to be integrated on. Historically this could be volume (VOL) or bound

In [ ]:
Wl_exact = 1/6
Wr_exact = -1/3

In [ ]:
"""grid_functions = []
L2Error = []
H1Error = []
Means = []
kmax = 8
ks = [i for i in range(1, kmax)]
Draw(mesh)"""